In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import FastICA  # Change to FastICA for ICA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbPipeline


In [2]:
# Set the decimal format
pd.options.display.float_format = "{:.2f}".format

# Read the data
df = pd.read_csv("diabetes_prediction_dataset.csv")

# Handle duplicates
df = df.drop_duplicates()

# Remove unnecessary values [0.00195%]
df = df[df['gender'] != 'Other']

In [3]:
# Recategorize smoking history
def recategorize_smoking(smoking_status):
    if smoking_status in ['never', 'No Info']:
        return 'non-smoker'
    elif smoking_status == 'current':
        return 'current'
    elif smoking_status in ['ever', 'former', 'not current']:
        return 'past_smoker'

df['smoking_history'] = df['smoking_history'].apply(recategorize_smoking)


In [4]:
# One-hot encoding
def perform_one_hot_encoding(df, column_name):
    dummies = pd.get_dummies(df[column_name], prefix=column_name)
    df = pd.concat([df.drop(column_name, axis=1), dummies], axis=1)
    return df

data = df.copy()
data = perform_one_hot_encoding(data, 'gender')
data = perform_one_hot_encoding(data, 'smoking_history')


In [5]:
# Resampling
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)

# Split data into features and target variable
X = data.drop('diabetes', axis=1)
y = data['diabetes']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Define numerical and categorical features
numerical_features = X.select_dtypes(include='number').columns
categorical_features = X.select_dtypes(include='object').columns

# ICA for numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('ica', FastICA(n_components=min(X_train[numerical_features].shape[0], X_train[numerical_features].shape[1])))
])

# One-hot encoding for categorical features
categorical_transformer = OneHotEncoder()

# Combine transformers using ColumnTransformer
preprocessor_ica = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the pipeline with ICA
clf_ica = imbPipeline(steps=[
    ('preprocessor_ica', preprocessor_ica),
    ('over', over),
    ('under', under),
    ('classifier', DecisionTreeClassifier(
        max_depth=8,
        min_samples_split=0.2,
        min_samples_leaf=2,
        criterion='gini'
    ))
])


In [7]:
# Train the model with ICA
clf_ica.fit(X_train, y_train)


Pipeline(steps=[('preprocessor_ica',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('ica',
                                                                   FastICA(n_components=6))]),
                                                  Index(['age', 'hypertension', 'heart_disease', 'bmi', 'HbA1c_level',
       'blood_glucose_level'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index([], dtype='object'))])),
                ('over', SMOTE(sampling_strategy=0.1)),
                ('under', RandomUnderSampler(sampling_strategy=0.5)),
                ('classifier',
                 DecisionTreeClassifier(max_depth=8, min_samples_leaf=2,
                                        min_samples_split=0.2))])

In [8]:
# Predict on the test set
y_pred_ica = clf_ica.predict(X_test)

# Evaluate the model with ICA
print("Model Accuracy with ICA: ", accuracy_score(y_test, y_pred_ica))
print(classification_report(y_test, y_pred_ica))

Model Accuracy with ICA:  0.9036721106834495
              precision    recall  f1-score   support

           0       0.97      0.93      0.95     17525
           1       0.47      0.66      0.55      1701

    accuracy                           0.90     19226
   macro avg       0.72      0.79      0.75     19226
weighted avg       0.92      0.90      0.91     19226

